In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn import *
import matplotlib.pyplot as plt # to better format confusion matrix
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('../data/all_deaths_clean.csv', encoding='utf-8')

In [47]:
nan_counts = df.isna().sum()
percentages = round(df.isna().mean() * 100, 1)
null_values = pd.concat([nan_counts, percentages], axis=1, keys=["count", "%"])

print("NaN Value Percentages by Column: \n")
print(null_values.sort_values(by=["%"]))


NaN Value Percentages by Column: 

                       count     %
id                         1   0.0
state                      0   0.0
county                     0   0.0
jail                       0   0.0
year                       0   0.0
date_of_death            282   3.7
gender                   389   5.1
cause_short              576   7.6
race                     672   8.9
custody_status          1251  16.5
first_name              1361  18.0
last_name               1361  18.0
cause_detail            1474  19.5
cause_detail_group      1474  19.5
date_incarcerated       1909  25.2
length_incarceration    1913  25.3
dob                     3915  51.7
age                     4246  56.1
mid_name                5775  76.3
full_name               6293  83.1
what_info_from_media    7253  95.8
num_media_descriptors   7253  95.8
yob                     7355  97.1
suffix                  7504  99.1
race_detail             7535  99.5


In [79]:

# df_model_data = df[["id", "year","yob","age","gender","num_media_descriptors","length_incarceration","cause_detail_group"]]
df_model_data = df[["id", "state", "county", "jail", "year", "gender", "cause_short", "race", "cause_detail_group", "length_incarceration"]]
df_model_data_remove_nans = df_model_data.apply(lambda x:x.fillna(x.value_counts().index[0]))

data = df_model_data_remove_nans.drop(columns=["gender"])
target = df_model_data_remove_nans["gender"]

train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.7, random_state=999)

train_categorical = OneHotEncoder().fit_transform(train_data[["state","county", "jail", "cause_short", "race", "cause_detail_group"]].values).toarray()
train_numerical = train_data.drop(columns = ["state","county", "jail", "cause_short", "race", "cause_detail_group"], axis = 1).values

# combining numerical and categorical data
joined_data = np.append(train_categorical, train_numerical, axis=1)
# print(len(test_data))
# print(len(train_data))
# print(len(train_target))
print(len(joined_data))

test_categorical = OneHotEncoder().fit_transform(test_data[["state", "county", "jail", "cause_short", "race", "cause_detail_group"]].values).toarray()
test_numerical = test_data.drop(columns = ["state", "county", "jail", "cause_short", "race", "cause_detail_group"], axis = 1).values

logistic_model = LogisticRegression(max_iter = 1000)
logistic_model.fit(X = joined_data, y = train_target)

# print(len(joined_data))
# print(len(train_target))

predict_input = np.append(test_categorical, test_numerical, axis=1)
# print(len(test_categorical))
# print(len(test_numerical))
print(len(predict_input))

predict_output = logistic_model.predict(predict_input)


ConfusionMatrixDisplay.from_estimator(logistic_model, predict_input, test_target)



2271
5300


ValueError: X has 924 features, but LogisticRegression is expecting 824 features as input.